# **Vertexai Init**

In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [2]:
PROJECT_ID = "gen-lang-client-0341374211"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = PROJECT_ID
    LOCATION = LOCATION

    # Initialize Vertex AI
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
search_query = """Sea food near Googleplex
1600 Amphitheatre Parkway
Mountain View, CA 94043
United States"""
#'how to make a great pastrami sandwich'

# **Realtime Google Search with Langchain**

In [9]:
!pip install -U duckduckgo_search
!python3 -m pip install googlesearch-python
!pip install -q langchain playwright beautifulsoup4 html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=e3b88fff1a57fc1b8dcc09f393b3251db16506a051dcc89aac2fc2e5c025bfe3
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.2/613.2 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta

In [10]:
search_query = 'Sea food near Googleplex\n1600 Amphitheatre Parkway\nMountain View, CA 94043\nUnited States'

In [11]:
google_search_results = []
structured_response = []

In [12]:
number_of_results = 2
from googlesearch import search
results = search(search_query, lang="en", num_results=number_of_results)

In [13]:
for result in results:
  if not result.startswith("https://www.tripadvisor.com"):
    google_search_results.append(result)

In [14]:
google_search_results

['https://www.yelp.com/search?cflt=seafood&find_loc=Mountain+View%2C+CA+94043',
 'https://www.yelp.com/search?cflt=seafood&find_loc=Mountain+View%2C+CA',
 'https://www.opentable.com/cuisine/best-seafood-restaurants-mountain-view-ca']

In [15]:
import html2text
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer

async def do_webscraping(link):
    try:
        urls = [link]
        loader = AsyncHtmlLoader(urls)
        docs = loader.load()

        html2text_transformer = Html2TextTransformer()
        docs_transformed = html2text_transformer.transform_documents(docs)

        if docs_transformed != None and len(docs_transformed) > 0:
            metadata = docs_transformed[0].metadata
            title = metadata.get('title', '')
            return {
                'summary': docs_transformed[0].page_content,
                'title': title,
                'metadata': metadata,
                'clean_content': html2text.html2text(docs_transformed[0].page_content)
            }
        else:
            return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [16]:
for link in google_search_results:
  print(link)
  response = await do_webscraping(link)
  if response != None:
    structured_response.append(response)

https://www.yelp.com/search?cflt=seafood&find_loc=Mountain+View%2C+CA+94043


Fetching pages: 100%|##########| 1/1 [00:02<00:00,  2.36s/it]


https://www.yelp.com/search?cflt=seafood&find_loc=Mountain+View%2C+CA


Fetching pages: 100%|##########| 1/1 [00:02<00:00,  2.13s/it]


https://www.opentable.com/cuisine/best-seafood-restaurants-mountain-view-ca


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.59s/it]


In [17]:
structured_response

[{'summary': 'Yelp\n\nYelp for Business\n\nWrite a Review\n\nLog InSign Up\n\nRestaurants\n\nDelivery\n\nBurgers\n\nChinese\n\nItalian\n\nReservations\n\nJapanese\n\nMexican\n\nThai\n\nHome Services\n\nContractors\n\nElectricians\n\nHome Cleaners\n\nHVAC\n\nLandscaping\n\nLocksmiths\n\nMovers\n\nPlumbers\n\nAuto Services\n\nAuto Repair\n\nAuto Detailing\n\nBody Shops\n\nCar Wash\n\nCar Dealers\n\nOil Change\n\nParking\n\nTowing\n\nMore\n\nDry Cleaning\n\nPhone Repair\n\nBars\n\nNightlife\n\nHair Salons\n\nGyms\n\nMassage\n\nShopping\n\nMore\n\nFilters\n\n$$$$$$$$$$\n\nSuggested\n\nOpen Now\n\n\\--:--\n\nOffers Delivery\n\nReservations\n\nFree Wi-Fi\n\nOutdoor Seating\n\nDogs Allowed\n\nFeatures\n\nOffers Takeout\n\nGood for Groups\n\nGood for Dinner\n\nGood for Kids\n\nSee all\n\nDistance\n\nBird\'s-eye View\n\nDriving (5 mi.)\n\nBiking (2 mi.)\n\nWalking (1 mi.)\n\nWithin 4 blocks\n\nYelpRestaurantsSeafood\n\n# The Best 10 Seafood Restaurants near Mountain View, CA 94043\n\nSort:Recom

# **Information Extraction**

In [18]:
import re
import json

def extract_json(input_string):
    # Extract JSON within ``` block
    matches = re.findall(r'```(.*?)```', input_string, re.DOTALL)

    if matches:
        # Join the matches into a single string
        json_content = ''.join(matches)

        # Remove periods
        json_content = re.sub(r'\.', '', json_content)

        return json_content
    else:
        print("No ``` block found.")
        return None

In [19]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

def execute_prompt(prompt, max_output_tokens=8192):
  model = GenerativeModel("gemini-pro")
  responses = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": max_output_tokens,
        "temperature": 0,
        "top_p": 1
    },
  stream=True,
  )

  final_response = []

  for response in responses:
      final_response.append(response.candidates[0].content.parts[0].text)

  return ".".join(final_response)

In [20]:
def get_text_extract_prompt(title, summary):
  prompt = f"""
  Here is its title: {title}
  Here is some text extracted:
  ---------
  {summary}
  ---------

  Web pages can have a lot of useless junk in them.
  For example, there might be a lot of ads, or a
  lot of navigation links, or a lot of text that
  is not relevant to the topic of the page. We want
  to extract only the useful information from the text.

  You can use the url and title to help you understand
  the context of the text.
  Please extract only the useful information from the text.
  Try not to rewrite the text, but instead extract
  only the useful information from the text.
  """
  return prompt

In [21]:
summarries = []

In [22]:
for structured_response_item in structured_response:
    title = structured_response_item['title']
    summary = structured_response_item['summary']
    if summary != "<html><body></body></html>":
      print(f'Summary for Title: {title}\n')
      text_extract_prompt = get_text_extract_prompt(title, summary)
      prompt_response = execute_prompt(text_extract_prompt)
      summarries.append(prompt_response)

Summary for Title: THE BEST 10 Seafood Restaurants near MOUNTAIN VIEW, CA 94043 - Last Updated December 2023 - Yelp

Summary for Title: THE BEST 10 Seafood Restaurants in MOUNTAIN VIEW, CA - Last Updated December 2023 - Yelp

Summary for Title: 15 Best Seafood Restaurants In Mountain View | OpenTable



In [23]:
summarries

['1. Limón: Peruvian seafood and cocktail bar with a delicious menu of mouth.-watering seafood and land animal options.\n2. Pacific Catch: Seafood,. tacos, and sushi bar with outdoor seating and Korean-style seafood pancakes.\n3. The Sea by Alexander’s Steakhouse: Seafood, steakhouse,. and bar with fresh seafood and jumbo shrimp.\n4. Cap’t Loui: Seafood, fish & chips, and Cajun/Creole with free parking. and large group friendly options.\n5. Rustic House Oyster Bar and Grill - Los Altos: Seafood, bar, and American with a grouper special and outdoor seating.\n6. The City Fish: Sandwiches, seafood, and fish &. chips with big portions of quality seafood at a reasonable price.\n7. King’s Fish House - San Jose: Seafood with outdoor seating and a tartare sauce with a great balance of spicy and seafood taste.\n8. La M.area of the Sea: Seafood food stand with fresh oysters and great condiments.\n9. Supreme Crab: Seafood and Cajun/Creole with sports on TV and large group friendly options.\n10. G

# **Retrieval-augmented generation (RAG)**

In [24]:
!pip install -U -q google.generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.9/146.9 kB 2.8 MB/s eta 0:00:00


In [25]:
%pip install -Uq chromadb pydantic typing-extensions==4.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 

In [26]:
import textwrap
import chromadb
import numpy as np
import pandas as pd
from google.colab import userdata

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings
genai.configure(api_key=userdata.get('google_key'))

In [27]:
collection_name = 'localstore'

In [28]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [29]:
# Set up the DB
db = create_chroma_db(summarries, collection_name)

In [30]:
pd.DataFrame(db.peek(3))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.059009771794080734, -0.04740039259195328, -...",None,1. Limón: Peruvian seafood and cocktail bar wi...,None,None
1,1,"[0.05023900046944618, -0.054701533168554306, -...",None,1. Limón: Peruvian seafood and cocktail bar wi...,None,None
2,2,"[0.03670496493577957, -0.06272248923778534, -0...",None,- Vaso Azzurro is a cherished neighborhood gem...,None,None


In [31]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [32]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [33]:
query = "where can I get seafood"
passage = get_relevant_passage(query, db)
Markdown(passage)
prompt = make_prompt(query, passage)
Markdown(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'where can I get seafood'
  PASSAGE: '1. Limón: Peruvian seafood and cocktail bar with a focus on fresh seafood. and mouth-watering dishes. 2. Pacific Catch: Seafood restaurant offering tacos., sushi, and poke bowls, with a focus on sustainable and healthy seafood options. 3. Cap’t Loui: Cajun/Creole seafood restaurant. known for its generous portions of fresh seafood and its friendly atmosphere. 4. Rustic House Oyster Bar and Grill: Seafood restaurant with a focus on oysters and other. fresh seafood dishes, as well as a variety of grilled meats. 5. The City Fish: Seafood restaurant known for its large portions of quality seafood at a reasonable price, including sandwiches, fish and chips, and other seafood dishes. .6. La Marea of the Sea: Seafood food stand offering fresh oysters and other seafood dishes. 7. The Sea by Alexander’s Steakhouse: Seafood restaurant with a focus on fresh seafood dishes, including a variety of grilled and. roasted seafood options. 8. King’s Fish House: Seafood restaurant with a focus on fresh seafood dishes, including a variety of grilled and roasted seafood options, as well as a raw bar. 9. Gochi: Japanese restaurant with a focus on fresh seafood dishes, including sushi, sashimi, and other. seafood dishes. 10. Forthright Oyster Bar & Kitchen: Seafood restaurant with a focus on oysters and other fresh seafood dishes, as well as a variety of grilled meats and other dishes.'

    ANSWER:
  

In [34]:
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)
Markdown(answer.text)

1. Limón
2. Pacific Catch
3. Cap’t Loui
4. Rustic House Oyster Bar and Grill
5. The City Fish
6. La Marea of the Sea
7. The Sea by Alexander’s Steakhouse
8. King’s Fish House
9. Gochi
10. Forthright Oyster Bar & Kitchen

# **Trulens Custom LLM Evaluation**

**Please make sure to run the Retrieval-augmented generation (RAG) to set the Db**

In [ ]:
!pip install litellm
!pip install trulens_eval==0.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.0
    Uninstalling typing_extensions-4.6.0:
      Successfully uninstalled typing_extensions-4.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<

**Note - You must restart the runtime in order to use newly installed versions.**

In [ ]:
from IPython.display import JSON
from trulens_eval import Feedback, Tru
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [ ]:
from google.cloud import aiplatform
aiplatform.init(
    project = PROJECT_ID,
    location= LOCATION
)

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('google_key')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import litellm
litellm.vertex_project = PROJECT_ID
litellm.vertex_location = LOCATION

In [ ]:
from trulens_eval.feedback.provider.litellm import LiteLLM
litellm_provider = LiteLLM()

In [ ]:
import litellm
from trulens_eval.tru_custom_app import instrument

class Gemini_RAG:
    def get_relevant_passage(self, query):
      passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
      return passage

    def make_prompt(query, relevant_passage):
      escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
      prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
      If the passage is irrelevant to the answer, you may ignore it.
      QUESTION: '{query}'
      PASSAGE: '{relevant_passage}'

        ANSWER:
      """).format(query=query, relevant_passage=escaped)

      return prompt

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        response = litellm.completion(model="gemini-pro", messages= [
            {"role": "user",
            "content":
            f"We have provided context information below. \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please answer the question: {query}"
            }
        ])

        content = ''
        if response != None and len(response.choices) > 0:
          content = response.choices[0].message.content
          print(content)

        return content

    @instrument
    def query(self, query: str) -> str:
        passage = self.get_relevant_passage(query)
        Markdown(passage)
        prompt = make_prompt(query, passage)
        Markdown(prompt)
        completion = self.generate_completion(query, prompt)
        return completion

# Please make sure to run the Retrieval-augmented generation (RAG) to set the Db
rag = Gemini_RAG()

In [ ]:
import numpy as np
from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.feedback import GroundTruthAgreement, Groundedness

# Define groundedness
grounded = Groundedness(groundedness_provider=litellm_provider)
f_groundedness = Feedback(grounded.groundedness_measure, name = "Groundedness").on(
    TruLlama.select_source_nodes().node.text # context
).on_output().aggregate(grounded.grounded_statements_aggregator)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(litellm_provider.relevance, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(litellm_provider.qs_relevance, name = "Context Relevance").on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.mean)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [ ]:
app_id = 'Personally.AIStore'

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = app_id,
    feedbacks = [f_groundedness, f_qa_relevance])

In [ ]:
# Need to check with Trulens team as the library is expecting OpenAI Key
# OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')

In [ ]:
with tru_rag as recording:
    rag.query("where can I get seafood")

1. Limón
2. Pacific Catch
3. The Sea by Alexander’s Steakhouse
4. Cap’t Loui
5. Rustic House Oyster Bar and Grill
6. The City Fish
7. Cook’s Seafood
8. King’s Fish House
9. La Marea of the Sea
10. Supreme Crab
Could not locate app.query.rets.source_nodes[:].node.text in app/record.


ERROR:concurrent.futures:exception calling callback for <Future at 0x7b0f3ef88d00 state=finished raised RuntimeError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/trulens_eval/feedback/feedback.py", line 669, in extract_selection
    arg_vals[k] = list(q_within_o.get(o))
  File "/usr/local/lib/python3.10/dist-packages/trulens_eval/utils/serial.py", line 851, in get
    for start_selection in start_items:
  File "/usr/local/lib/python3.10/dist-packages/trulens_eval/utils/serial.py", line 851, in get
    for start_selection in start_items:
  File "/usr/local/lib/python3.10/dist-packages/trulens_eval/utils/serial.py", line 851, in get
    for start_selection in start_items:
  File "/usr/local/lib/python3.10/dist-packages/trulens_eval/utils/serial.py", line 852, in get
    for last_selection in last_step.get(start_selection):
  File "/usr/local/lib/python3.10/dist-packages/trulens_eval/utils/serial.py", line 321, in get
    for r in self.get(obj=obj[0]

In [ ]:
tru.get_leaderboard(app_ids=[app_id])

,latency,total_cost
app_id,,
Personally.AIStore,7.5,0.0


In [ ]:
tru.get_records_and_feedback(app_ids=[app_id])

(               app_id                                           app_json  \
 0  Personally.AIStore  {"app_id": "Personally.AIStore", "tags": "-", ...   
 
                    type                                     record_id  \
 0  Gemini_RAG(__main__)  record_hash_1bb3999e0cd224c3c5f80eb44e67a5c9   
 
                        input  \
 0  "where can I get seafood"   
 
                                               output tags  \
 0  "1. Lim\u00f3n\n2. Pacific Catch\n3. The Sea b...    -   
 
                                          record_json  \
 0  {"record_id": "record_hash_1bb3999e0cd224c3c5f...   
 
                                            cost_json  \
 0  {"n_requests": 0, "n_successful_requests": 0, ...   
 
                                            perf_json  \
 0  {"start_time": "2023-12-22T02:27:07.369211", "...   
 
                            ts  latency  total_tokens  total_cost  
 0  2023-12-22T02:27:15.792313        8             0         0.0  ,
 [])